In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import statsmodels.formula.api as smf

In [2]:
idh = pd.read_csv('data/idh_estados_2021.csv', encoding='utf-8', decimal=',')
votacao = pd.read_csv('data/votacao_partido_munzona_2022_BRASIL.csv', encoding='utf-8')
vacina = pd.read_csv('data/cobertura_vacinal_municipios_2023.csv', encoding='utf-8')

## ❓Questão 1: IDH x Cobertura Vacinal

Existe relação entre IDH e cobertura vacinal dos estados? Qual?

In [3]:
vacina_estado = vacina.groupby('UF')['COBERTURA_VACINAL'].mean().reset_index()

idh_vacina = pd.merge(idh, vacina_estado, on='UF')

idh_vacina_sorted = idh_vacina.sort_values('IDHM2021')

idh_vacina_sorted['IDHM2021'] = pd.to_numeric(idh_vacina_sorted['IDHM2021'], errors='coerce')

idh_vacina_sorted['COBERTURA_VACINAL'] = pd.to_numeric(idh_vacina_sorted['COBERTURA_VACINAL'], errors='coerce')

fig1 = px.scatter(
    idh_vacina_sorted,
    x='IDHM2021',
    y='COBERTURA_VACINAL',
    hover_name='UF',
    labels={'IDHM2021': 'IDH (2021)', 'COBERTURA_VACINAL': 'Cobertura Vacinal (%)'},
    title='Relação entre IDH dos estados e Cobertura Vacinal da Influenza em 2023',
)

# Add manual trendline
x_vals = idh_vacina_sorted['IDHM2021'].values
y_vals = idh_vacina_sorted['COBERTURA_VACINAL'].values

# Calculate linear regression manually
z = np.polyfit(x_vals, y_vals, 1)
p = np.poly1d(z)

# Add trendline to the plot
fig1.add_scatter(
    x=x_vals,
    y=p(x_vals),
    mode='lines',
    name='Trendline',
    line=dict(color='red', width=2)
)

fig1.update_layout(
    xaxis=dict(categoryorder='array', categoryarray=idh_vacina_sorted['IDHM2021']),
)

fig1.show()


## ❓Questão 2: Votação x Cobertura Vacinal

Municípios com alta votação em candidatos de partidos com discurso antivacina em 2022 apresentaram menor índice de vacinação em 2023?

In [4]:
# Filtra apenas o 1º turno
votacao_1 = votacao[votacao['TURNO'] == 1]

# Soma votos por município e partido
votos_municipio = votacao.groupby(['MUNICIPIO', 'UF', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS_VALIDOS'].sum().reset_index()

# Seleciona o partido mais votado por município
mais_votado = votos_municipio.sort_values('QT_VOTOS_NOMINAIS_VALIDOS', ascending=False).drop_duplicates(['MUNICIPIO', 'UF'])


partidos_antivacina = ['PL', 'PTB', 'PSC', 'PP', 'REPUBLICANOS']  # você pode ajustar

# Cria uma coluna indicando se o partido mais votado no município é antivacina
mais_votado['ANTIVACINA'] = mais_votado['SG_PARTIDO'].isin(partidos_antivacina)


df = pd.merge(vacina, mais_votado, on=['MUNICIPIO','UF'], how='inner')

media_por_grupo = df.groupby('ANTIVACINA')['COBERTURA_VACINAL'].mean()

In [5]:
# Traduz booleanos para rótulos mais amigáveis
df['Partido_Antivacina'] = df['ANTIVACINA'].map({True: 'Antivacina', False: 'Outros'})

fig = px.box(
    df,
    x='Partido_Antivacina',
    y='COBERTURA_VACINAL',
    color='Partido_Antivacina',
    hover_data=['MUNICIPIO', 'SG_PARTIDO'],
    title='Cobertura vacinal em 2023 por tipo de partido mais votado no município',
    labels={'COBERTURA_VACINAL': 'Cobertura Vacinal (%)', 'Partido_Antivacina': '', 'MUNICIPIO': 'Município'},
    points="all",  # mostra os pontos individuais
    color_discrete_map={'Antivacina': 'red', 'Outros': 'green'}
)

fig.update_layout(showlegend=False)
fig.show()

In [6]:
fig = px.scatter(
    df,
    x='UF',
    y='COBERTURA_VACINAL',
    color='UF',
    hover_data=['MUNICIPIO', 'SG_PARTIDO'],
    title='Cobertura vacinal por estado e tipo de partido mais votado (municípios)',
    labels={'COBERTURA_VACINAL': 'Cobertura Vacinal (%)', 'UF': 'Estado'},
    color_discrete_map={'Antivacina': 'red', 'Outros': 'green'}
)

fig.show()


## ❓Questão 3: Votos x IDH

Quais os três partidos mais votados nos cinco estados com maior IDH e nos cinco com menor IDHQual a distribuição dos votos válidos por partido nos estados com maior IDH?

In [7]:
# 1. Ordenar estados por IDH
top5_idh = idh.sort_values('IDHM2021', ascending=False).head(5)['UF'].tolist()
bottom5_idh = idh.sort_values('IDHM2021', ascending=True).head(5)['UF'].tolist()


In [8]:
# 2. Filtrar apenas o 1º turno da votação
votacao_1t = votacao[votacao['TURNO'] == 1]


In [9]:
# 3. Agrupar por estado e partido, somando votos
votos_agrupados = (
    votacao_1t
    .groupby(['UF', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS_VALIDOS']
    .sum()
    .reset_index()
)


In [10]:
def top3_por_estado(lista_estados):
    return (
        votos_agrupados[votos_agrupados['UF'].isin(lista_estados)]
        .sort_values(['UF', 'QT_VOTOS_NOMINAIS_VALIDOS'], ascending=[True, False])
        .groupby('UF')
        .head(3)
    )

# Top 3 por estado com maior IDH
top3_mais_idh = top3_por_estado(top5_idh)

# Top 3 por estado com menor IDH
top3_menos_idh = top3_por_estado(bottom5_idh)


In [11]:
fig1 = px.bar(
    top3_mais_idh,
    x='UF',
    y='QT_VOTOS_NOMINAIS_VALIDOS',
    color='SG_PARTIDO',
    title='Top 3 partidos mais votados nos 5 estados com maior IDH',
    text='SG_PARTIDO'
)
fig1.show()

fig2 = px.bar(
    top3_menos_idh,
    x='UF',
    y='QT_VOTOS_NOMINAIS_VALIDOS',
    color='SG_PARTIDO',
    title='Top 3 partidos mais votados nos 5 estados com menor IDH',
    text='SG_PARTIDO'
)
fig2.show()


In [12]:
# Função que recebe DataFrame com top 3 por estado e agrupa votos por partido para o grupo de estados
def votos_por_partido_agrupado(df_top3):
    return (
        df_top3
        .groupby('SG_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS']
        .sum()
        .reset_index()
        .sort_values('QT_VOTOS_NOMINAIS_VALIDOS', ascending=False)
    )

# Agrupa votos dos top 3 partidos dos 5 estados com maior IDH
votos_mais_idh = votos_por_partido_agrupado(top3_mais_idh)

fig_pizza_mais = px.pie(
    votos_mais_idh,
    values='QT_VOTOS_NOMINAIS_VALIDOS',
    names='SG_PARTIDO',
    title='Distribuição dos votos dos 3 partidos mais votados nos 5 estados com maior IDH',
    hole=0.4
)
fig_pizza_mais.show()

# Agrupa votos dos top 3 partidos dos 5 estados com menor IDH
votos_menos_idh = votos_por_partido_agrupado(top3_menos_idh)

fig_pizza_menos = px.pie(
    votos_menos_idh,
    values='QT_VOTOS_NOMINAIS_VALIDOS',
    names='SG_PARTIDO',
    title='Distribuição dos votos dos 3 partidos mais votados nos 5 estados com menor IDH',
    hole=0.4
)
fig_pizza_menos.show()


## ❓Questão 4: Votos x Cobertura Vacinal

Quais partidos foram mais votados nos estados com menor cobertura vacinal? E nos com maior?

In [13]:
# Média cobertura vacinal por estado
media_vacina_estado = vacina.groupby('UF')['COBERTURA_VACINAL'].mean().reset_index()

top5_vacina = media_vacina_estado.sort_values('COBERTURA_VACINAL', ascending=False).head(5)['UF'].tolist()
bottom5_vacina = media_vacina_estado.sort_values('COBERTURA_VACINAL', ascending=True).head(5)['UF'].tolist()

votacao_1t = votacao[votacao['TURNO'] == 1]

votos_agrupados = (
    votacao_1t
    .groupby(['UF', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS_VALIDOS']
    .sum()
    .reset_index()
)

def top3_partidos_por_estados(lista_estados):
    return (
        votos_agrupados[votos_agrupados['UF'].isin(lista_estados)]
        .sort_values(['UF', 'QT_VOTOS_NOMINAIS_VALIDOS'], ascending=[True, False])
        .groupby('UF')
        .head(3)
    )

top3_maior_vacina = top3_por_estado(top5_vacina)
top3_menor_vacina = top3_por_estado(bottom5_vacina)



In [14]:
import plotly.express as px

def plot_pizza_top3(df, titulo):
    votos_por_partido = (
        df.groupby('SG_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS']
        .sum()
        .reset_index()
        .sort_values('QT_VOTOS_NOMINAIS_VALIDOS', ascending=False)
    )
    fig = px.pie(
        votos_por_partido,
        names='SG_PARTIDO',
        values='QT_VOTOS_NOMINAIS_VALIDOS',
        title=titulo,
        hole=0.4
    )
    fig.show()

plot_pizza_top3(top3_maior_vacina, 'Top 3 partidos nos estados com MAIOR cobertura vacinal')
plot_pizza_top3(top3_menor_vacina, 'Top 3 partidos nos estados com MENOR cobertura vacinal')


In [15]:
def radar_partidos(df, titulo):
    votos_agrupados = df.groupby('SG_PARTIDO')['QT_VOTOS_NOMINAIS_VALIDOS'].sum()
    partidos = votos_agrupados.index.tolist()
    votos = votos_agrupados.values.tolist()

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=votos,
        theta=partidos,
        fill='toself',
        name=titulo
    ))

    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, max(votos)*1.1]
        )),
      showlegend=True,
      title=titulo
    )

    fig.show()

radar_partidos(top3_maior_vacina, 'Perfil dos 3 partidos mais votados - Estados com Maior Cobertura Vacinal')
radar_partidos(top3_menor_vacina, 'Perfil dos 3 partidos mais votados - Estados com Menor Cobertura Vacinal')


In [16]:
# 1. Média de cobertura vacinal por estado
media_vacinal_estado = vacina.groupby('UF')['COBERTURA_VACINAL'].mean().reset_index()
media_vacinal_estado.rename(columns={'COBERTURA_VACINAL': 'media_cobertura_vacinal'}, inplace=True)

# 2. Filtrar 1º turno e remover BR
votacao_1t = votacao[(votacao['TURNO'] == 1)]

# 3. Partido mais votado por estado
votos_estado = votacao_1t.groupby(['UF', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS_VALIDOS'].sum().reset_index()
partido_mais_votado = votos_estado.loc[votos_estado.groupby('UF')['QT_VOTOS_NOMINAIS_VALIDOS'].idxmax()]

# 4. Mesclar os dois
dados_plot = pd.merge(media_vacinal_estado, partido_mais_votado, on='UF', how='inner')

# 5. Ordenar pelo valor da cobertura vacinal
dados_plot_sorted = dados_plot.sort_values('media_cobertura_vacinal')


In [17]:
fig = px.bar(
    dados_plot_sorted,
    x='UF',
    y='media_cobertura_vacinal',
    color='SG_PARTIDO',
    hover_data=['QT_VOTOS_NOMINAIS_VALIDOS'],
    labels={
        'UF': 'Estado',
        'media_cobertura_vacinal': 'Cobertura Vacinal Média (%)',
        'SG_PARTIDO': 'Partido mais votado'
    },
    title='Cobertura Vacinal Média por Estado e Partido mais Votado no 1º Turno (2022)'
)

fig.update_layout(xaxis={'categoryorder': 'array', 'categoryarray': dados_plot_sorted['UF']})
fig.show()

In [18]:
# 1. Calcular média de cobertura vacinal por estado
media_vacinal_estado = vacina.groupby('UF')['COBERTURA_VACINAL'].mean().reset_index()
media_vacinal_estado.rename(columns={'COBERTURA_VACINAL': 'media_cobertura_vacinal'}, inplace=True)

# 2. Filtrar apenas 1º turno e remover 'BR'
votacao_1t = votacao[(votacao['TURNO'] == 1)]

# 3. Obter o partido mais votado por estado
votos_estado = votacao_1t.groupby(['UF', 'SG_PARTIDO'])['QT_VOTOS_NOMINAIS_VALIDOS'].sum().reset_index()
partido_mais_votado = votos_estado.loc[votos_estado.groupby('UF')['QT_VOTOS_NOMINAIS_VALIDOS'].idxmax()]

# 4. Mesclar
dados_mapa = pd.merge(media_vacinal_estado, partido_mais_votado, on='UF', how='inner')

# 5. Criar mapa
fig = px.choropleth(
    dados_mapa,
    geojson="https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/brazil-states.geojson",
    locations='UF',
    featureidkey="properties.sigla",  # 'sigla' ou 'UF' dependendo do GeoJSON
    color='media_cobertura_vacinal',
    color_continuous_scale="Viridis",
    hover_name='UF',
    hover_data={
        'media_cobertura_vacinal': ':.2f',
        'SG_PARTIDO': True,
        'QT_VOTOS_NOMINAIS_VALIDOS': True
    },
    labels={
        'media_cobertura_vacinal': 'Cobertura Vacinal Média (%)',
        'SG_PARTIDO': 'Partido mais votado',
        'QT_VOTOS_NOMINAIS_VALIDOS': 'Votos no Partido'
    },
    title='Cobertura Vacinal Média por Estado e Partido mais Votado no 1º Turno (2022)'
)

fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":50,"l":0,"b":0})
fig.show()


## ❓Questão 5: IDH x Votos x Cobertura Vacinal

Estatísticamente, podemos afirmar que de fato há relação entre IDH, partidos mais votados e cobertura vacinal?

In [19]:
# Média da cobertura por IDH
df['IDH'] = df['UF'].map(idh.set_index('UF')['IDHM2021'])
df.groupby('ANTIVACINA')['COBERTURA_VACINAL'].mean()

df[['IDH', 'COBERTURA_VACINAL']].corr()


,IDH,COBERTURA_VACINAL
IDH,1.000000,-0.211297
COBERTURA_VACINAL,-0.211297,1.000000


In [20]:
# Preparar variáveis
df['ANTIVACINA_NUM'] = df['ANTIVACINA'].astype(int)

# Modelo linear com interação
modelo = smf.ols('COBERTURA_VACINAL ~ IDH + ANTIVACINA_NUM + IDH:ANTIVACINA_NUM', data=df).fit()

print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:      COBERTURA_VACINAL   R-squared:                       0.287
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     48.77
Date:                Thu, 26 Jun 2025   Prob (F-statistic):               0.00
Time:                        16:47:13   Log-Likelihood:                -23504.
No. Observations:                5502   AIC:                         4.710e+04
Df Residuals:                    5456   BIC:                         4.740e+04
Df Model:                          45                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

In [21]:
# Certifique-se que IDH está numérico contínuo (float)
df['IDH_cont'] = df['IDH'].astype(float)

# Variável binária para partido antivacina
df['ANTIVACINA_NUM'] = df['ANTIVACINA'].astype(int)


In [22]:
import statsmodels.formula.api as smf

modelo_cont = smf.ols('COBERTURA_VACINAL ~ IDH_cont + ANTIVACINA_NUM + IDH_cont:ANTIVACINA_NUM', data=df).fit()

print(modelo_cont.summary())


                            OLS Regression Results                            
Dep. Variable:      COBERTURA_VACINAL   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     156.0
Date:                Thu, 26 Jun 2025   Prob (F-statistic):           4.65e-97
Time:                        16:47:13   Log-Likelihood:                -24210.
No. Observations:                5502   AIC:                         4.843e+04
Df Residuals:                    5498   BIC:                         4.845e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

In [23]:
import plotly.express as px

fig = px.scatter(
    df,
    x='IDH_cont',
    y='COBERTURA_VACINAL',
    color=df['ANTIVACINA'].map({True: 'Partido Antivacina', False: 'Outros'}),
    trendline='ols',
    labels={'IDH_cont': 'IDH Contínuo', 'COBERTURA_VACINAL': 'Cobertura Vacinal (%)'},
    title='Cobertura vacinal vs IDH contínuo por grupo de partido'
)
fig.show()
